# Class Attendance using Face Recognition (Haar Cascade and LBPH Algorithm)

<h2>Introduction</h2>

<h2>Haar Cascade</h2>

<h2>LBPH Algorithm</h2>

In [9]:
# !pip install opencv-python
# !pip install opencv-contrib-python --user
# !pip install opencv-contrib-python --upgrade

In [10]:
import cv2
import pandas as pd
import os
import numpy as np
import time
import datetime

In [11]:
def faceDetection(test_img):
    gray_img=cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    face_haar_cascade=cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')
    faces=face_haar_cascade.detectMultiScale(gray_img,scaleFactor=1.32,minNeighbors=5)

    return faces,gray_img

In [12]:
#Given a directory below function returns part of gray_img which is face alongwith its label/ID
def labels_for_training_data(directory):
    faces=[]
    faceID=[]

    for path,subdirnames,filenames in os.walk(directory):
        for filename in filenames:
            if filename.startswith("."):
                print("Skipping system file")#Skipping files that startwith .
                continue

            id=os.path.basename(path)#fetching subdirectory names
            img_path=os.path.join(path,filename)#fetching image path
            print("img_path:",img_path)
            print("id:",id)
            test_img=cv2.imread(img_path)#loading each image one by one
            if test_img is None:
                print("Image not loaded properly")
                continue
            faces_rect,gray_img=faceDetection(test_img)#Calling faceDetection function to return faces detected in particular image
            if len(faces_rect)!=1:
                continue #Since we are assuming only single person images are being fed to classifier
            (x,y,w,h)=faces_rect[0]
            roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from grayscale image
            faces.append(roi_gray)
            faceID.append(int(id))
    return faces,faceID

In [13]:
def train_classifier(faces,faceID):
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(faces,np.array(faceID))
    return face_recognizer

In [14]:
def draw_rect(test_img,face):
    (x,y,w,h)=face
    cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=5)

In [15]:
def put_text(test_img,text,x,y):
    cv2.putText(test_img,text,(x,y),cv2.FONT_HERSHEY_DUPLEX,2,(255,0,0),4)

---

In [16]:
#This module captures images via webcam and performs face recognition

faces,faceID= labels_for_training_data('trainingImages')
face_recognizer= train_classifier(faces,faceID)
face_recognizer.write('trainingData.yml')

# face_recognizer=cv2.face.LBPHFaceRecognizer_create()
# face_recognizer.read('trainingData.yml')#use this to load training data for subsequent runs

name = {0 : "Lita",1 : "Fafilia"}

cap=cv2.VideoCapture(0)

df = pd.read_csv("Student"+os.sep+"daftar_student.csv")
col_names = ['Id', 'Name', 'Date', 'Time']
attendance = pd.DataFrame(columns=col_names)

while True:
    ret,test_img=cap.read()
    faces_detected,gray_img=faceDetection(test_img)


    for face in faces_detected:
        (x,y,w,h)=face
        roi_gray=gray_img[y:y+w, x:x+h]
        id_,confidence=face_recognizer.predict(roi_gray)
        print("confidence:",confidence)
        print("id:",id_)
        draw_rect(test_img,face)
        predicted_name=name[id_]
        if confidence < 50:
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
            timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
            attendance.loc[len(attendance)] = [id_, predicted_name, date, timeStamp]
            put_text(test_img,predicted_name,x,y)
        else if confidence > 50:
            id_ = 'Unknown'
            name_ = str(id_)
            noOfFile = len(os.listdir("Unknown"))+1
            cv2.imwrite("Unknown"+os.sep+"Image"+str(noOfFile) +".jpg", test_img[y:y+h, x:x+w])
            put_text(test_img,name_,x,y)
    attendance = attendance.drop_duplicates(subset=['Id'], keep='first')
 
    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('face recognition',resized_img)
    if cv2.waitKey(1) == ord('q'):
        break

ts = time.time()
date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
Hour, Minute, Second = timeStamp.split(":")
fileName = "Attendance"+os.sep+"Attendance_"+date+"_"+Hour+"-"+Minute+"-"+Second+".csv"
attendance.to_csv(fileName, index=False)

cap.release()
cv2.destroyAllWindows

print("Attendance Successfull")

Skipping system file
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.31 (1).jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.31 (2).jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.31 (3).jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.31 (4).jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.31.jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.32 (1).jpeg
id: 0
img_path: trainingImages\0\WhatsApp Image 2020-06-04 at 15.04.32.jpeg
id: 0
img_path: trainingImages\1\emma-55b9c74bfa9ee14b4a1e13f29501e2be_600x400.jpg
id: 1
img_path: trainingImages\1\Emma-Watson140422.jpg
id: 1
img_path: trainingImages\1\emma-watson_gettyimages-619546914jpg.jpg
id: 1
img_path: trainingImages\1\EMMA_WATSON.jpg
id: 1
img_path: trainingImages\1\Emma_Watson_2013.jpg
id: 1
img_path: trainingImages\1\fikt6voiw411n51z2i8r.jpg
id: 1
img_path: trainingImages\1\images.jpg
id: 1
img_path: 

confidence: 44.84839669645604
id: 0
confidence: 44.84839669645604
id: 0
confidence: 46.34938620983055
id: 0
confidence: 47.66813916834865
id: 0
confidence: 47.66813916834865
id: 0
confidence: 45.07855742742508
id: 0
confidence: 45.46903471922828
id: 0
confidence: 45.46903471922828
id: 0
confidence: 45.316286410328104
id: 0
confidence: 43.89966258458054
id: 0
confidence: 43.89966258458054
id: 0
confidence: 44.98297760182213
id: 0
confidence: 44.680609029130686
id: 0
confidence: 44.680609029130686
id: 0
confidence: 48.48102505539961
id: 0
confidence: 48.09505965285588
id: 0
confidence: 48.09505965285588
id: 0
confidence: 50.0375928613424
id: 0
confidence: 46.38681958987811
id: 0
confidence: 46.38681958987811
id: 0
confidence: 45.72964348111058
id: 0
confidence: 46.118366613410494
id: 0
confidence: 46.118366613410494
id: 0
confidence: 48.51372213331553
id: 0
confidence: 46.126938093295365
id: 0
confidence: 46.126938093295365
id: 0
confidence: 46.17666304482964
id: 0
confidence: 45.5232523

confidence: 43.78366373594857
id: 0
confidence: 43.78366373594857
id: 0
confidence: 46.804573418829854
id: 0
confidence: 46.02350152228504
id: 0
confidence: 46.02350152228504
id: 0
confidence: 47.536004336097136
id: 0
confidence: 44.271869136483296
id: 0
confidence: 44.271869136483296
id: 0
confidence: 44.09526111382687
id: 0
confidence: 45.96883768587947
id: 0
confidence: 45.96883768587947
id: 0
confidence: 48.078089409001755
id: 0
confidence: 47.77397558372238
id: 0
confidence: 47.77397558372238
id: 0
confidence: 46.894379395392875
id: 0
confidence: 45.43459004975111
id: 0
confidence: 45.43459004975111
id: 0
confidence: 46.87080397893697
id: 0
confidence: 45.42368227145233
id: 0
confidence: 45.42368227145233
id: 0
confidence: 44.6257094438017
id: 0
confidence: 44.933060674360284
id: 0
confidence: 44.933060674360284
id: 0
confidence: 47.88803382833262
id: 0
confidence: 46.72466416130033
id: 0
confidence: 46.72466416130033
id: 0
confidence: 44.73178471946007
id: 0
confidence: 45.827880

confidence: 45.79562229337417
id: 0
confidence: 45.38536001936719
id: 0
confidence: 45.38536001936719
id: 0
confidence: 50.301055140629614
id: 0
confidence: 49.0133138944192
id: 0
confidence: 49.0133138944192
id: 0
confidence: 49.05357681845163
id: 0
confidence: 47.03595484509193
id: 0
confidence: 47.03595484509193
id: 0
confidence: 47.932399880180036
id: 0
confidence: 46.85903258763898
id: 0
confidence: 46.85903258763898
id: 0
confidence: 46.00957902580922
id: 0
confidence: 45.251979234106884
id: 0
confidence: 45.251979234106884
id: 0
confidence: 47.830201397818904
id: 0
confidence: 46.44909822758079
id: 0
confidence: 46.44909822758079
id: 0
confidence: 47.06161562477079
id: 0
confidence: 47.82345614687392
id: 0
confidence: 47.82345614687392
id: 0
confidence: 46.3375142899554
id: 0
confidence: 45.32416057208799
id: 0
confidence: 45.32416057208799
id: 0
confidence: 46.98508728832678
id: 0
confidence: 46.83783662677296
id: 0
confidence: 46.83783662677296
id: 0
confidence: 50.80250277613